In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

# Load the dataset
df_results_tele = pd.read_csv('../../../data/outputs/results_tele.csv')

# List the companies in the dataset
companies = df_results_tele['Company'].unique()

# Available columns (metrics) to plot
available_columns = ['Percentage Change (%)', 'Quarterly Volume', 'Volatility', 'Quarter ROI (%)', 'ROI on $1500 ($)']


In [2]:
# Create the layout for the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    # Div to hold both dropdowns in a horizontal layout
    html.Div([
        # Dropdown for selecting companies (multi-select)
        dcc.Dropdown(
            id='company-filter-results',
            options=[{'label': company, 'value': company} for company in companies],
            value=companies,  # Initially select all companies
            multi=True,
            placeholder="Select companies",
            style={'width': '45%', 'display': 'inline-block'}  # Adjust width
        ),
        
        # Dropdown for selecting the metric
        dcc.Dropdown(
            id='metric-dropdown-results',
            options=[{'label': metric, 'value': metric} for metric in available_columns],
            value='Percentage Change (%)',  # Default to 'Percentage Change (%)'
            clearable=False,
            style={'width': '45%', 'display': 'inline-block', 'margin-left': '5%'}  # Adjust width and space
        )
    ], style={'display': 'flex', 'justify-content': 'center'}),  # Ensure the divs are centered
    
    # Graph to display the filtered data
    dcc.Graph(id='results-tele-graph')
])

# Callback to update the graph based on selected companies and metrics
@app.callback(
    Output('results-tele-graph', 'figure'),
    [Input('company-filter-results', 'value'), Input('metric-dropdown-results', 'value')]
)
def update_results_graph(selected_companies, selected_metric):
    # Filter the dataframe based on selected companies
    filtered_df = df_results_tele[df_results_tele['Company'].isin(selected_companies)]
    
    # Create the figure
    fig = go.Figure()

    # Add traces for each selected company
    for company in selected_companies:
        company_data = filtered_df[filtered_df['Company'] == company]
        fig.add_trace(go.Scatter(x=company_data['Quarter'], y=company_data[selected_metric],
                                 mode='lines', name=company))
    
    # Update the layout without a title
    fig.update_layout(
        xaxis_title="Quarter",
        yaxis_title=selected_metric,
        showlegend=True  # Keep the legend visible
    )
    
    return fig

# Run the app (disable debug mode to remove the blue circle)
if __name__ == '__main__':
    app.run_server(debug=False, port=8053)

OSError: Address 'http://127.0.0.1:8053' already in use.
    Try passing a different port to run_server.